In [1]:
# a quick cnn coded in tensorflow, via some wrapper functions

In [1]:
# import data and modules
import tensorflow as tf
import matplotlib.pyplot as plt
from mnist import MNIST


In [2]:
data = MNIST('./data/mnist')

In [3]:
num_train = data.num_train
num_test = data.num_test
img_size = data.img_size
num_classes = data.num_classes
num_channels = 1

In [4]:
num_train
num_test
img_size
num_classes
num_channels

55000

10000

28

10

1

In [5]:
# cnn architecture
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape=shape))


def new_conv_layer(input, filter_size, num_inputs, num_filters, stride_step, pool_step, use_pooling):
    
    filter_shape = [filter_size, filter_size, num_inputs, num_filters]
    
    layer = tf.nn.conv2d(input=input, 
                         filter=new_weights(filter_shape),
                         strides=[1, stride_step, stride_step, 1],
                         padding='SAME')
    
    if use_pooling:
        layer = tf.nn.max_pool(layer, [1, 2, 2, 1], strides=[1, pool_step, pool_step, 1], padding='SAME')

    layer = tf.nn.relu(layer)
    
    return layer

def flatten_layer(input):

    input_shape = input.shape[1:4]
    num_features = input_shape.num_elements()
    layer = tf.reshape(input, [-1, num_features], name='flatten_layer')
        
    return layer, num_features

def new_fc_layer(input, num_inputs, num_outputs):
    
    weights = tf.Variable(tf.truncated_normal(shape=[num_inputs, num_outputs]))
    biases = tf.Variable(tf.truncated_normal(shape=[num_outputs]))
    
    layer = tf.matmul(input, weights) + biases
    
    return layer
    
def new_dropout_layer(input, rate, training_bool_flag):
    
    layer = tf.layers.dropout(input=input, rate=rate, training=training_bool_flag)
    
    return layer
        
        

In [6]:
# placeholders for data 

x = tf.placeholder(dtype=tf.float32, shape=[None, img_size, img_size, num_channels])
y_true = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])
y_true_cls = tf.argmax(y_true, axis=1)



In [21]:
filter_size1 = 3
num_filters1 = 16

filter_size2 = 5
num_filters2 = 32

fc_size1 = 128

conv1 = new_conv_layer(input=x, filter_size=filter_size1, num_inputs=num_channels, num_filters=num_filters1, 
                       stride_step=1, pool_step=2, use_pooling=True)
conv2 = new_conv_layer(input=conv1, filter_size=filter_size2, 
                       num_inputs=num_filters1, num_filters=num_filters2, 
                       stride_step=1, pool_step=2, use_pooling=True)

flat_layer1, flat_layer1_features = flatten_layer(conv2)

fc1 = new_fc_layer(input=flat_layer1, num_inputs=flat_layer1_features, num_outputs=fc_size1)
logits = new_fc_layer(input=fc1, num_inputs=fc_size1, num_outputs=num_classes)

ypred = tf.nn.softmax(logits=logits)
ypred_cls = tf.argmax(ypred, axis=1)



In [22]:
# optimizer 
loss = tf.nn.softmax_cross_entropy_with_logits(labels=x, logits=logits)
# import ipdb; ipdb.set_trace()

cost = tf.reduce_mean(loss)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)


In [25]:
logits.get_shape()
y_true.get_shape()
cost.get_shape()
loss.get_shape()

TensorShape([Dimension(None), Dimension(10)])

TensorShape([Dimension(None), Dimension(10)])

TensorShape([])

TensorShape([Dimension(None)])